In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
link2="predictions_2023-07-29T02.csv"
link1="results_analysis_v1.csv"
from sentence_transformers import SentenceTransformer, util
import torch
device= 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
from tqdm import tqdm

In [ ]:
import torch
from datetime import datetime
act_df = pd.read_csv(link2)
res_df = pd.read_csv(link1)

In [ ]:
model_feed=res_df[['medium_description_pim','HS_Code_x','Val-NoVal']]

In [ ]:
model_feed_val= model_feed[model_feed['Val-NoVal']=='validated']

# One-Hot-encoder

In [1]:
from sklearn.preprocessing import OneHotEncoder
df_for_onehot =model_feed_val['HS_Code_x'].value_counts().reset_index()
df_for_onehot['index_3']=df_for_onehot.index
model_feed_val2=model_feed_val.merge(df_for_onehot, left_on= 'HS_Code_x', right_on='index', how='inner')
y= model_feed_val2['index_3']
onhe=OneHotEncoder(handle_unknown='ignore', sparse=False).fit(y.to_numpy().reshape(-1,1))

NameError: name 'model_feed_val' is not defined

Import datasets

In [ ]:
from datasets import load_dataset
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, DataCollatorWithPadding

In [ ]:
Dataset.from_pandas(model_feed_val2)
raw_ds=DatasetDict({'train':
    Dataset.from_pandas(model_feed_val2)
           
} )

split_ds=raw_ds['train'].train_test_split(test_size=0.15, shuffle=True)
checkpoint= 'distilbert-base-cased'
tokenizer= AutoTokenizer.from_pretrained(checkpoint)
def tokenize_func(batch):
    return tokenizer(batch['medium_description_pim'], truncation=True)
token_ds= split_ds.map(tokenize_func, batched=True)
data_collator= DataCollatorWithPadding(tokenizer=tokenizer)
token_ds=token_ds.remove_columns(['medium_description_pim','Val-NoVal','__index_level_0__', 'HS_Code_x_x','HS_Code_x_y', 'index'])
token_ds=token_ds.rename_column('index_3','labels')

# Create DATALOADER

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
train_loader= DataLoader(
token_ds['train'],
    shuffle=True,
    batch_size=32,
    collate_fn=data_collator
)
valid_loader= DataLoader(
token_ds['test'],
    shuffle=False,
    batch_size=35,
    collate_fn=data_collator
)

In [ ]:
for batch in train_loader:
    for i, k in batch.items():
        print(i)
        print(k.shape)
    break

# Build the Model

In [ ]:
from utils_transformer import *

In [ ]:
model_customs= Encoder_Smash(
vocab_size= tokenizer.vocab_size,
    max_len= tokenizer.max_model_input_sizes[checkpoint],
    d_k= 16,
    d_model=64,
    n_heads=4,
    n_layers=2,
    n_classes=351,
    dropout_prob=0.2
)
model_customs.to(device)

# Optimizer & Train

In [ ]:
criterion= torch.nn.MultiLabelSoftMarginLoss()
optimizer= torch.optim.Adam(model_customs.parameters())

In [ ]:
epochs=70
train_losses= np.zeros(epochs)
test_losses= np.zeros(epochs)
best_valid_loss= np.Inf

for iter1 in range(epochs):
    model_customs.train()
    train_loss=0.0
    n_train=0.0
    for batch in tqdm(train_loader):
        batch= {k: v.to(device) for k, v in batch.items()}
        the_number=batch['labels']
        the_number2=np.array(the_number.cpu()).reshape(-1,1)
        torched_labels= torch.tensor(onhe.transform(the_number2)).to(device)
        optimizer.zero_grad()
        outputs= model_customs(batch['input_ids'], batch['attention_mask'])
        loss= criterion(outputs, torched_labels)
        loss.backward()
        optimizer.step()

        train_loss+= loss.item()*batch['input_ids'].size(0)
        n_train += batch['input_ids'].size(0)

    train_loss= train_loss/n_train
    #Now eval
    model_customs.eval()
    test_loss= 0.0
    n_test= 0.0

    for batch in tqdm(valid_loader):
        batch= {k: v.to(device) for k, v in batch.items()}
        the_number=batch['labels']
        the_number2=np.array(the_number.cpu()).reshape(-1,1)
        torched_labels= torch.tensor(onhe.transform(the_number2)).to(device)
        outputs_test= model_customs(batch['input_ids'], batch['attention_mask'])
        loss= criterion(outputs_test, torched_labels)
        test_loss += loss.item()*batch['input_ids'].size(0)
        n_test += batch['input_ids'].size(0)
    test_loss= test_loss/n_test

    if test_loss< best_valid_loss:
        torch.save(model_customs.state_dict(), 'best_model.pt')
        best_valid_loss= test_loss
        print('Saved weights')
        print(iter1, train_loss, test_loss)
    print(iter1, train_loss, test_loss)
    train_losses[iter1]= train_loss
    test_losses[iter1]= test_loss

In [ ]:
trans_encoder2.load_state_dict(torch.load('best_model.pt'))

# Validation Time

In [ ]:
n_correct= 0.0
n_correct_ver2= 0.0
n_total=0.0
output_list=[]
output_list_by_batch=[]
for batch in train_loader:
    
    batch = {k: v.to(device) for k, v in batch.items()}
    the_number=batch['labels']
    the_number2=np.array(the_number.cpu()).reshape(-1,1)
    torched_labels= torch.tensor(onhe.transform(the_number2)).to(device)
    outputs= model_customs(batch['input_ids'], batch['attention_mask'])
    to_list=np.array(outputs.detach().cpu())
    output_list_by_batch.append(to_list)
    for single_out in to_list:
        output_list.append(single_out)
    _, predictions = torch.max(outputs,1)
    n_correct += (predictions==batch['labels']).sum().item()
    array3=np.array(torch.argmax(outputs, axis=1).detach().cpu())
    array4=np.array(torch.argmax(torched_labels, axis=1).detach().cpu())
    n_correct_ver2+= sum(array3==array4)
    n_total += batch['labels'].shape[0]
test_acc= n_correct/n_total
test_acc2=n_correct_ver2/n_total
print(f' Text acc: {test_acc}')

In [ ]:
test_acc2

In [ ]:
n_correct_ver2

In [ ]:
np.array(torch.argmax(outputs, axis=1).detach().cpu())